# Install and import libraries

In [1]:
!pip install tabula-py

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import tabula

# Convert pdf to dataframe

In [7]:
url = "https://machikanekun-ticket.jp/files/shop/shoplist.pdf"
dfs = tabula.read_pdf(url, stream=True , pages = '4')

In [8]:
dfs[0].head(5)

,全店(○)...全店舗共通券[紙] 中小(●)...中小店舗専用券[紙],デジ (★)...デジタル商品券[中小店舗専用券],Unnamed: 0,Unnamed: 1
0,利用可能な商品券,NaN,NaN,NaN
1,店舗名 業種,NaN,所在地,電話番号
2,全店,中小 デジ,NaN,NaN
3,豊中市北部(〒560)豊中・蛍池・千里,NaN,NaN,NaN
4,1 ジュエリーズイホウ その他の業種 〇,● ★,北緑丘1-4-10,06-6854-1005


In [10]:
dfs[0].tail(5)

,全店(○)...全店舗共通券[紙] 中小(●)...中小店舗専用券[紙],デジ (★)...デジタル商品券[中小店舗専用券],Unnamed: 0,Unnamed: 1
34,31 hair&nail&eye cherie シェリ 理美容 〇,● ★,東豊中町4-7-8-1,06-6152-5855
35,32 メガネの愛眼 東豊中店 小売業 その他 〇,NaN,東豊中町5-1-12,06-6841-7130
36,33 しらすや Shirasu 食料品(スーパー含む) 〇,● ★,東豊中町5-1-15,06-6842-7455
37,34 とんがらし 飲食店 〇,● ★,東豊中町5-12-1 西谷ハイツ1F,06-6846-0740
38,35 ヘアークリニック 髪切床 理美容 〇,● ★,東豊中町5-2-132-101,06-6848-3855


# 処理の流れ
1. ヘッダ行（最初の3行）と、不要な行（地域ごとのインデックスの行）を削除
2. 1列目を整形：連番の取得、店名を取得、カテゴリと〇の有無、分割うまくいっているかチェック
3. 2列目を整形：●と☆を分割
4. 3列目を整形：住所を取得、建物名などは分割

# 1. ヘッダ削除
ヘッダ行（最初の3行）と、不要な行（地域ごとのインデックスの行）を削除する

In [17]:
df_org = dfs[0]
df = df_org.iloc[3:,:]
df.head(5)

,全店(○)...全店舗共通券[紙] 中小(●)...中小店舗専用券[紙],デジ (★)...デジタル商品券[中小店舗専用券],Unnamed: 0,Unnamed: 1
3,豊中市北部(〒560)豊中・蛍池・千里,NaN,NaN,NaN
4,1 ジュエリーズイホウ その他の業種 〇,● ★,北緑丘1-4-10,06-6854-1005
5,2 金比羅製麺 北緑丘店 飲食店 〇,● ★,北緑丘2-8-6,06-6855-5612
6,3 島熊山グラーヴ 飲食店 〇,● ★,緑丘1-25-5,06-6848-1256
7,4 りんご整骨院 医療福祉 〇,● ★,緑丘1-27-6 アメニティ緑丘A,06-6842-2508


In [18]:
df = df[~df.iloc[:,2].isnull()]
df.head(5)

,全店(○)...全店舗共通券[紙] 中小(●)...中小店舗専用券[紙],デジ (★)...デジタル商品券[中小店舗専用券],Unnamed: 0,Unnamed: 1
4,1 ジュエリーズイホウ その他の業種 〇,● ★,北緑丘1-4-10,06-6854-1005
5,2 金比羅製麺 北緑丘店 飲食店 〇,● ★,北緑丘2-8-6,06-6855-5612
6,3 島熊山グラーヴ 飲食店 〇,● ★,緑丘1-25-5,06-6848-1256
7,4 りんご整骨院 医療福祉 〇,● ★,緑丘1-27-6 アメニティ緑丘A,06-6842-2508
8,5 フラワーアレンジメント教室・花雑貨販売ラグレーヌ 家具・雑貨 〇,● ★,緑丘1-31-2-201,090-5334-9939


# 2. １列目のデータを整形
連番の取得、店名を取得、カテゴリと〇の有無、分割うまくいっているかチェック

In [19]:
print(df.iloc[1,0].split(' ', 1)[0]) # 通し番号
print(df.iloc[1,0].split(' ', 1)[1].rsplit(' ', 2)[0]) # 店舗名
print(df.iloc[1,0].split(' ', 1)[1].rsplit(' ', 2)[1]) # 業態
print('〇' in df.iloc[1,0]) # 〇 全店

2
金比羅製麺 北緑丘店
飲食店
True


In [20]:
df.iloc[:,0].str.split(' ', 1, expand=True)[0][:5]

4    1
5    2
6    3
7    4
8    5
Name: 0, dtype: object

# 3. ２列目を整形：●と☆を分割

In [21]:
df.iloc[:,1].head()

4    ● ★
5    ● ★
6    ● ★
7    ● ★
8    ● ★
Name: デジ (★)...デジタル商品券[中小店舗専用券], dtype: object

In [22]:
df['全店'] = df.iloc[:,0].str.contains('〇')
df['全店'] = df['全店'].replace(True, '〇')
df['デジタル'] = df.iloc[:,1].str.contains('★')
df['デジタル'] = df['デジタル'].replace(True, '★')
df['中小'] = df.iloc[:,1].str.contains('●')
df['中小'] = df['中小'].replace(True, '●')

# 4. 3列目を整形
住所を取得、建物名などを分割

In [30]:
print(df.iloc[3,2].split(' ', 1)[0]) # 住所１
print(df.iloc[3,2].split(' ', 1)[1]) # 住所２

緑丘1-27-6
アメニティ緑丘A


In [55]:
print(df.iloc[1,3]) # 電話番号

070-3789-5350


# 関数化する

In [32]:
def delete_header(df, header_rows=3):
    return(df.iloc[header_rows:,:])

In [33]:
def extract_shopname(df):
    shopnames = []
    categories = []
    tmp_series = df.iloc[:,0].str.split(' ', 1, expand=True)[1].str.replace('〇', '').str.rstrip(' ')
    #print(tmp_series)
    
    for x in tmp_series:
        #print(x)
        if x.endswith('その他'): # フォーマットがカテゴリによって微妙に違う
            shopname = x.rsplit(' ', 2)[0].rstrip(' ')
            category = x.replace(shopname, '').lstrip(' ')
            #print(shopname, ',', category, 'True')
        else:
            shopname = x.rsplit(' ', 1)[0].rstrip(' ')
            category = x.replace(shopname, '').lstrip(' ')
            #print(shopname, ',', category, 'False')
    
        shopnames.append(shopname)
        categories.append(category)
    df['shopname'] = pd.Series(shopnames, index=df.index)
    df['category'] = pd.Series(categories, index=df.index)
    return df

In [34]:
df = extract_shopname(df)

In [35]:
df['Address1'] = '大阪府豊中市' + df.iloc[:,2].str.split(' ', 1, expand=True)[0]
df['Address2'] = df.iloc[:,2].str.split(' ', 1, expand=True)[1]
df['Phone'] = df.iloc[:,3]
df.head()

,全店(○)...全店舗共通券[紙] 中小(●)...中小店舗専用券[紙],デジ (★)...デジタル商品券[中小店舗専用券],Unnamed: 0,Unnamed: 1,全店,デジタル,中小,shopname,category,Address1,Address2,Phone
4,1 ジュエリーズイホウ その他の業種 〇,● ★,北緑丘1-4-10,06-6854-1005,〇,★,●,ジュエリーズイホウ,その他の業種,大阪府豊中市北緑丘1-4-10,None,06-6854-1005
5,2 金比羅製麺 北緑丘店 飲食店 〇,● ★,北緑丘2-8-6,06-6855-5612,〇,★,●,金比羅製麺 北緑丘店,飲食店,大阪府豊中市北緑丘2-8-6,None,06-6855-5612
6,3 島熊山グラーヴ 飲食店 〇,● ★,緑丘1-25-5,06-6848-1256,〇,★,●,島熊山グラーヴ,飲食店,大阪府豊中市緑丘1-25-5,None,06-6848-1256
7,4 りんご整骨院 医療福祉 〇,● ★,緑丘1-27-6 アメニティ緑丘A,06-6842-2508,〇,★,●,りんご整骨院,医療福祉,大阪府豊中市緑丘1-27-6,アメニティ緑丘A,06-6842-2508
8,5 フラワーアレンジメント教室・花雑貨販売ラグレーヌ 家具・雑貨 〇,● ★,緑丘1-31-2-201,090-5334-9939,〇,★,●,フラワーアレンジメント教室・花雑貨販売ラグレーヌ,家具・雑貨,大阪府豊中市緑丘1-31-2-201,None,090-5334-9939


In [36]:
df.iloc[-1,10:]

Address2            None
Phone       06-6848-3855
Name: 38, dtype: object

In [37]:
delete_header(df_org).head(5)

,全店(○)...全店舗共通券[紙] 中小(●)...中小店舗専用券[紙],デジ (★)...デジタル商品券[中小店舗専用券],Unnamed: 0,Unnamed: 1
3,豊中市北部(〒560)豊中・蛍池・千里,NaN,NaN,NaN
4,1 ジュエリーズイホウ その他の業種 〇,● ★,北緑丘1-4-10,06-6854-1005
5,2 金比羅製麺 北緑丘店 飲食店 〇,● ★,北緑丘2-8-6,06-6855-5612
6,3 島熊山グラーヴ 飲食店 〇,● ★,緑丘1-25-5,06-6848-1256
7,4 りんご整骨院 医療福祉 〇,● ★,緑丘1-27-6 アメニティ緑丘A,06-6842-2508


In [38]:
def process_df(df):
    df = delete_header(df)
    df = df[~df.iloc[:,2].isnull()] # 地域行削除

    df.index = df.iloc[:,0].str.split(' ', 1, expand=True)[0]
    df.index.name = '番号'

    df = extract_shopname(df)

    df['全店'] = df.iloc[:,0].str.contains('〇').fillna(False)
    df['全店'] = df['全店'].replace(True, '〇')
    df['全店'] = df['全店'].replace(False, '')

    df['デジタル'] = df.iloc[:,1].str.contains('★').fillna(False)
    df['デジタル'] = df['デジタル'].replace(True, '★')
    df['デジタル'] = df['デジタル'].replace(False, '')
    
    df['中小'] = df.iloc[:,1].str.contains('●').fillna(False)
    df['中小'] = df['中小'].replace(True, '●')
    df['中小'] = df['中小'].replace(False, '')
    
    df['住所１'] = '大阪府豊中市' + df.iloc[:,2].str.split(' ', 1, expand=True)[0]
    df['住所２'] = df.iloc[:,2].str.split(' ', 1, expand=True)[1].fillna('')
    df['電話番号'] = df.iloc[:,3]
    
    return df.iloc[:,4:]

In [44]:
df = process_df(df_org)
df

,shopname,category,全店,デジタル,中小,住所１,住所２,電話番号
番号,,,,,,,,
1812,アルチザン 服部店,10.小売業 その他,〇,★,●,大阪府豊中市服部元町1-7-3,,06-6866-0352
1813,eyelash nail salon ALO ROOMY 11. サービス業,理美容,〇,★,●,大阪府豊中市服部本町1-6-1,メゾンマルシン 2F,06-7709-1052
1814,ほっかほっか亭 服部店 3.小売業,飲食店,〇,★,●,大阪府豊中市服部本町2-2-3,,06-6862-1168
1815,ダイナーズカフェ ALOHASpirit 3. 小売業,飲食店,〇,★,●,大阪府豊中市服部本町2-2-7,1F,06-7165-6054
1816,セブン-イレブン 豊中服部南町4丁目店 5.小売業,コンビニエンスストア,〇,★,●,大阪府豊中市服部南町4-5-13,,06-6862-3316
1817,セブンイレブン豊中穂積1丁目店 5. 小売業,コンビニエンスストア,〇,★,●,大阪府豊中市穂積1-4-22,,06-6863-7611
1818,やまおか,10.小売業 その他,〇,,●,大阪府豊中市服部西町2-5-25,,06-6864-3291
1819,中国料理 桂林 3. 小売業,飲食店,〇,,●,大阪府豊中市服部西町2-5-3,,06-6864-2555
1820,B-ambitious 3. 小売業,飲食店,〇,★,●,大阪府豊中市服部豊町1-6-7,矢内ハイツ服部104号,06-6862-2072


# 関数実行して全データ処理
https://machikanekun-ticket.jp/files/shop/shoplist.pdf

In [45]:
url = "https://machikanekun-ticket.jp/files/shop/shoplist.pdf"
dfs = tabula.read_pdf(url, stream=True , pages = '4-50')

In [46]:
dfs_processed = []
for i, df_org in enumerate(dfs):
    dfs_processed.append(process_df(df_org))

In [47]:
df_processed = pd.concat(dfs_processed)

# 整形エラー部分をマニュアル修正
shopname が nan になっている行があるので修正、電話番号は使わないのでそっとしておく

In [57]:
df_processed[df_processed.shopname.isnull()]

,shopname,category,全店,デジタル,中小,住所１,住所２,電話番号
番号,,,,,,,,
396,NaN,小売業 その他,〇,★,●,大阪府豊中市玉井町1-1-1,エトレとよなか3F 06-4866-5020,メガネ本舗エトレとよなか店
397,NaN,サービス業 その他,〇,,●,大阪府豊中市玉井町1-1-1,エトレとよなか4F 06-6841-9192,アルペンクイックフィットネス エトレ豊中
398,NaN,飲食店,〇,★,●,大阪府豊中市玉井町1-1-1,エトレとよなか4F 06-6852-8633,エッグ・ラップ
399,NaN,飲食店,〇,★,●,大阪府豊中市玉井町1-1-1,エトレとよなか4F 06-6858-4373,おこのみ焼き ju:ju
400,NaN,飲食店,〇,,●,大阪府豊中市玉井町1-1-1,エトレとよなか4F 06-6854-8470,リュリュ阪急豊中店
...,...,...,...,...,...,...,...,...
1510,NaN,サービス業 その他,〇,,●,大阪府豊中市東寺内町13-5,06-6380-5299,やなぎ屋クリーニング 緑地東店
1511,NaN,電化製品,〇,★,●,大阪府豊中市東寺内町13-5,ライオンズマンション105 06-6318-5167,(有)小西電化 緑地公園店
1512,NaN,理美容,〇,★,●,大阪府豊中市東寺内町13-5-101,06-6369-2266,hair's effect


In [60]:
df_processed[df_processed.shopname.isnull()]['shopname'] = df_processed[df_processed.shopname.isnull()]['電話番号']

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [71]:
null_index = df_processed[df_processed.shopname.isna()].index
df_processed.loc[null_index,'shopname'] = df_processed.loc[null_index,'電話番号']

In [80]:
df_processed.loc[null_index, :]

,shopname,category,全店,デジタル,中小,住所１,住所２,電話番号
番号,,,,,,,,
396,メガネ本舗エトレとよなか店,小売業 その他,〇,★,●,大阪府豊中市玉井町1-1-1,エトレとよなか3F 06-4866-5020,メガネ本舗エトレとよなか店
397,アルペンクイックフィットネス エトレ豊中,サービス業 その他,〇,,●,大阪府豊中市玉井町1-1-1,エトレとよなか4F 06-6841-9192,アルペンクイックフィットネス エトレ豊中
398,エッグ・ラップ,飲食店,〇,★,●,大阪府豊中市玉井町1-1-1,エトレとよなか4F 06-6852-8633,エッグ・ラップ
399,おこのみ焼き ju:ju,飲食店,〇,★,●,大阪府豊中市玉井町1-1-1,エトレとよなか4F 06-6858-4373,おこのみ焼き ju:ju
400,リュリュ阪急豊中店,飲食店,〇,,●,大阪府豊中市玉井町1-1-1,エトレとよなか4F 06-6854-8470,リュリュ阪急豊中店
...,...,...,...,...,...,...,...,...
1510,やなぎ屋クリーニング 緑地東店,サービス業 その他,〇,,●,大阪府豊中市東寺内町13-5,06-6380-5299,やなぎ屋クリーニング 緑地東店
1511,(有)小西電化 緑地公園店,電化製品,〇,★,●,大阪府豊中市東寺内町13-5,ライオンズマンション105 06-6318-5167,(有)小西電化 緑地公園店
1512,hair's effect,理美容,〇,★,●,大阪府豊中市東寺内町13-5-101,06-6369-2266,hair's effect


In [83]:
df_processed.tail(95)

,shopname,category,全店,デジタル,中小,住所１,住所２,電話番号
番号,,,,,,,,
1737,ガンバ大阪オフィシャルショップ Blu SPAZIO,その他の業種,〇,,●,大阪府豊中市千里万博公園3-3,,06-6875-8558
1738,錦わらい イオンタウン豊中緑丘店 3. 小売業,飲食店,〇,★,●,大阪府豊中市緑丘4-1,イオンタウン豊中緑丘1F,06-4865-0065
1739,Green Parks topic 2.小売業,衣料品,〇,,,大阪府豊中市緑丘4-1-1,イオンタウン豊中緑丘,06-6151-9930
1740,ABC-MART イオンタウン豊中緑丘店 2.小売業,衣料品,〇,,,大阪府豊中市緑丘4-1-2,イオンタウン豊中緑丘SC1F,06-6844-5706
1741,からあげボンボン 3. 小売業,飲食店,〇,★,●,大阪府豊中市東豊中町4-3-8,,06-6398-9997
...,...,...,...,...,...,...,...,...
1827,幸せな花屋-SUN,10.小売業 その他,〇,★,●,大阪府豊中市中桜塚1-2-36,,080-3429-8344
1828,アルチザン 岡町店,10.小売業 その他,〇,★,●,大阪府豊中市中桜塚1-2-37,,06-6863-9360
1829,itanClip -前菜base- 岡町店 3.小売業,飲食店,〇,★,●,大阪府豊中市中桜塚2-20-11,2F,06-6848-1368


1738行目（今回追加された店舗）から、category 列のところのフォーマットが変わっていて、うまく抽出できていないけど、実用上は問題ないので、そっとしておく

In [84]:
df_processed.to_csv('./shoplist20211122.csv')